In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
font = {'size': 16}
matplotlib.rc('font', **font)
tqdm.tqdm.pandas()

In [2]:
%%time
main_data_df = pd.read_csv('./Prepped Data/main_data_baseline_20211202.csv')
print(main_data_df.shape)
main_data_df.head()

(5188101, 18)
Wall time: 10.2 s


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대
1,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대
2,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대
3,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대
4,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대


In [4]:
main_data_df.dtypes

시군구            object
번지             object
본번            float64
부번            float64
건물/단지명         object
거래유형           object
전용면적(㎡)       float64
계약년월            int64
계약일             int64
거래/보증금(만원)      int64
월세(만원)          int64
층             float64
건축년도          float64
도로명            object
부동산유형          object
가격범위           object
건물단지명_키워드      object
브랜드            object
dtype: object

In [5]:
def find_letters(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    
    x = str(x)
    
    for i in range(10):
        x = x.replace(str(i), '')
    
    x = x.replace('-', '')
    
    if x == '':
        return '글자없음'
    else:
        return x

In [6]:
main_data_df['글자'] = main_data_df['번지'].apply(find_letters)

In [7]:
main_data_df['글자'].value_counts()

글자없음    5180824
산          2409
BL         1553
가           560
Name: 글자, dtype: int64

In [11]:
main_data_df['글자'] = main_data_df['글자'].apply(lambda x: '산' if x == '산' else '글자없음')

In [3]:
%%time
coor_df = pd.read_csv('./Prepped Data/연속지적도출처_지번별좌표_본번부번정리_20211222.csv')
print(coor_df.shape)
coor_df.head()

(922318, 12)
Wall time: 1.25 s


,시군구명,법정동명,지번_지목,지번,지목,x좌표,y좌표,글자있음,글자,지번만,본번,부번
0,강남구,개포동,0-1 가,0-1,가,206331.014484,442028.661953,0,글자없음,0-1,0,1
1,강남구,개포동,0-2 가,0-2,가,206254.177141,441986.263641,0,글자없음,0-2,0,2
2,강남구,개포동,0-3 가,0-3,가,206174.409344,441932.356344,0,글자없음,0-3,0,3
3,강남구,개포동,0-4 가,0-4,가,206631.548237,442577.897175,0,글자없음,0-4,0,4
4,강남구,개포동,0-5 가,0-5,가,205243.406000,442547.328000,0,글자없음,0-5,0,5


In [8]:
coor_df['글자'].value_counts()

글자없음    905366
산        16951
도            1
Name: 글자, dtype: int64

In [9]:
coor_df['글자'] = coor_df['글자'].apply(lambda x: '산' if x == '산' else '글자없음')

In [10]:
coor_df['글자'].value_counts()

글자없음    905367
산        16951
Name: 글자, dtype: int64

In [12]:
def remove_first_and_last_spaces(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    
    x = str(x)
    
    if x[0] == ' ':
        x = x[1:]
    
    if x[-1] == ' ':
        x = x[:-1]
    
    return x

In [13]:
main_data_df['시군구'] = main_data_df['시군구'].apply(remove_first_and_last_spaces)

In [14]:
coor_df['시군구명'] = coor_df['시군구명'].apply(lambda x: x.replace(' ', ''))
coor_df['법정동명'] = coor_df['법정동명'].apply(lambda x: x.replace(' ', ''))

In [15]:
coor_df['시군구'] = '서울특별시 ' + coor_df['시군구명'] + ' ' + coor_df['법정동명']

In [18]:
main_data_df['본번'] = main_data_df['본번'].astype('Int64')
main_data_df['부번'] = main_data_df['부번'].astype('Int64')

In [21]:
coor_df['본번'] = coor_df['본번'].astype('Int64')
coor_df['부번'] = coor_df['부번'].astype('Int64')

In [19]:
main_data_df.head()

,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,글자
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대,글자없음
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대,글자없음
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대,글자없음
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,글자없음
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,글자없음


In [22]:
coor_df.head()

,시군구명,법정동명,지번_지목,지번,지목,x좌표,y좌표,글자있음,글자,지번만,본번,부번,시군구
0,강남구,개포동,0-1 가,0-1,가,206331.014484,442028.661953,0,글자없음,0-1,0,1,서울특별시 강남구 개포동
1,강남구,개포동,0-2 가,0-2,가,206254.177141,441986.263641,0,글자없음,0-2,0,2,서울특별시 강남구 개포동
2,강남구,개포동,0-3 가,0-3,가,206174.409344,441932.356344,0,글자없음,0-3,0,3,서울특별시 강남구 개포동
3,강남구,개포동,0-4 가,0-4,가,206631.548237,442577.897175,0,글자없음,0-4,0,4,서울특별시 강남구 개포동
4,강남구,개포동,0-5 가,0-5,가,205243.406000,442547.328000,0,글자없음,0-5,0,5,서울특별시 강남구 개포동


In [23]:
cols_to_keep = ['시군구', '지목', 'x좌표', 'y좌표', '글자', '본번', '부번']
coor_df = coor_df[cols_to_keep]
print(coor_df.shape)
coor_df.head()

(922318, 7)


,시군구,지목,x좌표,y좌표,글자,본번,부번
0,서울특별시 강남구 개포동,가,206331.014484,442028.661953,글자없음,0,1
1,서울특별시 강남구 개포동,가,206254.177141,441986.263641,글자없음,0,2
2,서울특별시 강남구 개포동,가,206174.409344,441932.356344,글자없음,0,3
3,서울특별시 강남구 개포동,가,206631.548237,442577.897175,글자없음,0,4
4,서울특별시 강남구 개포동,가,205243.406000,442547.328000,글자없음,0,5


In [26]:
coor_df = coor_df.drop_duplicates().reset_index(drop=True)
print(coor_df.shape)

(922318, 7)


In [24]:
main_data_df.head()

,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,글자
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대,글자없음
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대,글자없음
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대,글자없음
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,글자없음
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,글자없음


In [27]:
merge_df = main_data_df.merge(coor_df, on=['시군구', '본번', '부번', '글자'])
print(merge_df.shape)
merge_df.head()

(5097504, 22)


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,글자,지목,x좌표,y좌표
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대,글자없음,대,204626.688971,442141.82001
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대,글자없음,대,204626.688971,442141.82001
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대,글자없음,대,204626.688971,442141.82001
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,글자없음,대,204626.688971,442141.82001
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,글자없음,대,204626.688971,442141.82001


In [28]:
%%time
merge_df.to_csv('./Prepped Data/main_data_coor_20211223.csv', index=False)

Wall time: 29.1 s
